In [1]:
import os

os.chdir("./content")

lm_dir = "tf-levenberg-marquardt"
if not os.path.exists(lm_dir):
  !git clone https://github.com/fabiodimarco/$lm_dir

os.chdir(lm_dir)

Cloning into 'tf-levenberg-marquardt'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 53 (delta 31), reused 29 (delta 14), pack-reused 0
Receiving objects: 100% (53/53), 32.99 KiB | 5.50 MiB/s, done.
Resolving deltas: 100% (31/31), done.


In [2]:
!pip install --upgrade pip
!pip install tensorflow 
!pip install openpyxl

In [3]:
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.preprocessing import MaxAbsScaler

def peaks(grid):
    x = np.linspace(-2, 2, grid)
    y = np.linspace(-2, 2, grid)
    x, y = np.meshgrid(x, y)
    z = 3*(1-x)**2 * np.exp(-(x**2) - (y+1)**2) - 10*(x/5 - x**3 - y**5) * np.exp(-x**2 - y**2) - 1/3 * np.exp(-(x+1)**2 - y**2)
    return x, y, z

def create_dataframe(grid):
   x, y, z = peaks(grid)
   data = {'X': x.flatten(), 'Y': y.flatten(), 'Z': z.flatten()}
   df = pd.DataFrame(data)
   return df

In [4]:
df_25 = create_dataframe(grid=5)
df_1000 = create_dataframe(grid=32)

In [5]:
def split_df(df):
    _input = np.vstack([df['X'], df['Y']]).T
    _output = np.array(df['Z'])
    return (_input, _output)

In [6]:
import tensorflow as tf
import numpy as np
from keras import regularizers
from keras import initializers
import levenberg_marquardt as lm

# layers, neurons
class ShuffleArchitecture:
    def __init__(self, input_size, hidden_sizes, output_size, act_h, act_o, param_reg):
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.output_size = output_size
        self.act_h = act_h
        self.act_o = act_o
        self.regularizer = regularizers.L2(param_reg)
        self.initializer = initializers.RandomUniform(minval=-0.5, maxval=0.5, seed=np.random.randint(1, 10000))

    def set_architecture(self):
        self.model = tf.keras.Sequential()
        self.model.add(tf.keras.layers.Dense(self.hidden_sizes[0],
                        input_shape=(self.input_size,),
                        activation=self.act_h,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,                        
                        ))  # input layer
        
        for size in self.hidden_sizes[1:]:  # hidden layers
            self.model.add(tf.keras.layers.Dense(size,
                            activation=self.act_h,
                            kernel_regularizer=self.regularizer,
                            kernel_initializer=self.initializer,  
                        ))

        self.model.add(tf.keras.layers.Dense(self.output_size,
                        activation=self.act_o,
                        kernel_regularizer=self.regularizer,
                        kernel_initializer=self.initializer,  
                        ))  # output layer

    def create_model(self, _learning_rate):
        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=_learning_rate),
            loss=tf.keras.losses.MeanSquaredError())

        self.lm_model = lm.ModelWrapper(
            tf.keras.models.clone_model(self.model))

        self.lm_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=_learning_rate),
            loss=lm.MeanSquaredError())
        return(self.lm_model)

2024-02-07 11:39:37.303603: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-07 11:39:39.182475: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-07 11:39:39.182581: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-07 11:39:39.457721: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-07 11:39:40.064970: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-07 11:39:40.065977: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [7]:
import tensorflow as tf
import matplotlib.pyplot as plt
import time
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.metrics import r2_score, mean_squared_error


class TrainWithSmallDataset:
    def __init__(self, batch_size=1000, store=25):
        self.batch_size = batch_size
        self.betters = []
        self.store = store

    def create_dataset(self, input, output):
      input = tf.expand_dims(tf.cast(input, tf.float32), axis=-1)
      output = tf.expand_dims(tf.cast(output, tf.float32), axis=-1)

      dataset = tf.data.Dataset.from_tensor_slices((input, output))
      dataset = dataset.shuffle(len(input))
      dataset = dataset.batch(self.batch_size).cache()
      dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
      return (dataset, input, output)

    def split_dataset(self, input, output, sup_input, sup_output):

      input_train, input_vt, output_train, output_vt = train_test_split(input, output, test_size=0.3, shuffle = True)
      input_val, input_test, output_val, output_test = train_test_split(input_vt, output_vt, test_size=0.5, shuffle = True)

      self.train_dataset, self.train_input, self.train_output = self.create_dataset(input_train, output_train)
      self.val_dataset, self.val_input, self.val_output = self.create_dataset(input_val, output_val)
      self.test_dataset, self.test_input, self.test_output = self.create_dataset(input_test, output_test)
      self.vt_dataset, self.vt_input, self.vt_output = self.create_dataset(input_vt, output_vt)
      self.sup_dataset, self.sup_input, self.sup_output = self.create_dataset(sup_input, sup_output)
      self.dataset, self.input, self.output = self.create_dataset(input, output)

      self._train = (input_train, output_train)
      self._vt = (input_vt, output_vt)
      self._val = (input_val, output_val)
      self._test = (input_test, output_test)
 
    def train_using_lm(self, train_dataset, epochs=1000):
      early_stopping_monitor = EarlyStopping(monitor='val_loss',
                                              patience=6,
                                              restore_best_weights=True)

      t2_start = time.perf_counter()
      self.results = self.lm_model.fit(train_dataset,
                                            epochs=epochs,
                                            validation_data=self.val_dataset,
                                            callbacks=[early_stopping_monitor],
                                            verbose=0)
      t2_stop = time.perf_counter()
      print("Elapsed time: ", t2_stop - t2_start)
      print ("Stopped at epoch: ", early_stopping_monitor.stopped_epoch)

    def get_metrics(self):
          self.sup_prediction = self.lm_model.predict(self.sup_input)
          self.test_prediction = self.lm_model.predict(self.test_input)

          pred = self.lm_model.predict(self.input).flatten()
          test_pred = self.test_prediction.flatten()
          val_pred = self.lm_model.predict(self.val_input).flatten()
          vt_pred = self.lm_model.predict(self.vt_input).flatten()
          sup_pred = self.sup_prediction.flatten()

          r2 = r2_score(self.output, pred)
          r2_test = r2_score(self.test_output, test_pred)
          r2_val = r2_score(self.val_output, val_pred)
          r2_vt = r2_score(self.vt_output, vt_pred)
          r2_sup = r2_score(self.sup_output, sup_pred)

          mse = mean_squared_error(self.output, pred)
          mse_test = mean_squared_error(self.test_output, test_pred)
          mse_val = mean_squared_error(self.val_output, val_pred)
          mse_vt = mean_squared_error(self.vt_output, vt_pred)
          mse_sup = mean_squared_error(self.sup_output, sup_pred)

          metrics = {
                          'r2': r2,
                          'r2_sup': r2_sup,
                          'r2_test': r2_test,
                          'r2_val': r2_val,
                          'r2_vt': r2_vt,
                          'mse': mse,
                          'mse_sup': mse_sup,
                          'mse_test': mse_test,
                          'mse_val': mse_val,
                          'mse_vt': mse_vt
                          }

          return metrics

    def plot_results(self):
      fig = plt.figure(figsize=(18, 6))
      ax0 = fig.add_subplot(1, 3, 1, projection='3d')
      ax2 = fig.add_subplot(1, 3, 2, projection='3d')
      ax1 = fig.add_subplot(1, 3, 3)

      ax0.set_title('Superficie Fit')
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_prediction, c='r', marker='s', label="lm")
      ax0.scatter(self.sup_input[:, 0], self.sup_input[:, 1], self.sup_output, c='b', marker='o', label="reference")
      ax0.legend()

      ax2.set_title('Teste Fit')
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_prediction, c='r', marker='s', label="lm")
      ax2.scatter(self.test_input[:, 0], self.test_input[:, 1], self.test_output, c='b', marker='o', label="reference")
      ax2.legend()

      ax1.plot(self.results.history['loss'], label='Erro Treino')
      ax1.plot(self.results.history['val_loss'], label='Erro validação')
      ax1.set_title('Histórico de Treinamento')
      ax1.set_ylabel('Função de Custo')
      ax1.set_xlabel('Épocas de Treinamento')
      ax1.legend(['Erro Treino', 'Erro validação'])

      return(fig)

In [8]:
import pickle
from itertools import product

class Tester:
  def __init__(self, run_times=500, dataset_run_times=10):
    self.run_times = run_times
    self.better_metrics = {}
    self.dataset_run_times = dataset_run_times
    self.input_25, self.output_25 = split_df(df_25)
    self.input_1000, self.output_1000 = split_df(df_1000)
  
  def setArchitecure(self, trainer, _hidden_sizes, _pg, _lr):
    shuffler = ShuffleArchitecture(input_size=2,
                                    hidden_sizes=_hidden_sizes,
                                    output_size=1,
                                    act_h='tanh',
                                    act_o='linear',
                                    param_reg=_pg)
    shuffler.set_architecture()    
    trainer.lm_model = shuffler.create_model(_lr)

  def Train(self, trainer, epochs=1000):
    trainer.train_using_lm(trainer.train_dataset, epochs=epochs)
    return(trainer.get_metrics(), trainer.lm_model)

  def SaveModelWeights(self, model, fileName):
    path = f"../models/{fileName}.keras"
    open(path,'w').close()
    model.save_weights(path)
  
  def SaveFigs(self, trainer, fileName):
    path = f"../results/{fileName}.png" 
    open(path,'w').close()
    figure = trainer.plot_results()
    figure.savefig(path)
    plt.close(figure)

  def SaveDataset(self, trainer, fileName):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'wb') as f:
      pickle.dump((trainer._train, trainer._vt, trainer._val, trainer._test), f)
  
  def LoadDataset(self, fileName, trainer=None):
    path = f"../dataset/{fileName}.pkl" 
    with open(path, 'rb') as f:
      _train, _vt, _val, _test = pickle.load(f)
    trainer._train, trainer._vt, trainer._val, trainer._test = _train, _vt, _val, _test

  def LoopWeights(self, sort_by, boundarie, trainer, idx):
    better_model = 0
    save = False

    for i in range(self.run_times):
      print (f"+++++++++++ [{idx}] | {i + 1} ++++++++++++++++++")
      metrics, model = self.Train(trainer)
      if (metrics[sort_by] <= boundarie): # should be >= to acsending metrics
        fileName = f"model_{idx}_{better_model}"
        self.SaveModelWeights(model, fileName)
        self.SaveFigs(trainer, fileName)
        self.better_metrics[fileName] = metrics
        better_model += 1
        save = True
    
    return(save)

  def Loop(self, sort_by, boundarie, hidden_sizes, regularizers, learning_rate):
    trainer = TrainWithSmallDataset()

    for count, (hidden_size, reg, lr) in enumerate(product(hidden_sizes, regularizers, learning_rate), start=1):
      header =  f"Hidden Size={hidden_size}, regularizer={reg}, learning_rate={lr}"
      print(f"Testando combinacao{count}: {header}")
      self.setArchitecure(trainer, hidden_size, reg, lr)
      for j in range(self.dataset_run_times):
        trainer.split_dataset(self.input_25, self.output_25, self.input_1000, self.output_1000)
        if (self.LoopWeights(sort_by, boundarie, trainer, f"{count}_{j}") == True):
          self.SaveDataset(trainer, f"dataset_{count}_{j}")
          self.DisplayBetterResults('mse_sup', header, f"{count}_{j}")
        self.better_metrics = {}

  def DisplayBetterResults(self, sort_by, header, dataset=0):
    df = pd.DataFrame.from_dict(self.better_metrics, orient='index')
    df = df.sort_values([sort_by])
    display(df)
    path = f'../results/metrics_{dataset}'
    df.to_excel(f"{path}.xlsx", index=True)
    print(f"DataFrame salvo em {path}")
    with open(f"{path}.txt", 'w') as arquivo:
      arquivo.write(header)



# Treinando com 25 dados


In [9]:
tester = Tester(run_times=25, dataset_run_times=10)

In [10]:
tester.Loop(sort_by='mse_sup',
            boundarie=2.0,
            hidden_sizes = [[5, 6], [2, 7], [6, 3]],
            regularizers=[0.02, 0.2, 0.5],
            learning_rate=[0.01, 0.1])

Testando combinacao1: Hidden Size=[5, 6], regularizer=0.02, learning_rate=0.01
+++++++++++ [1_0] | 1 ++++++++++++++++++


Elapsed time:  1.1270985350001865
Stopped at epoch:  11
1/1 [==============================] - 0s 19ms/step
+++++++++++ [1_0] | 2 ++++++++++++++++++
Elapsed time:  0.2168090260001918
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_0] | 3 ++++++++++++++++++
Elapsed time:  0.1612379090001923
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 4 ++++++++++++++++++
Elapsed time:  0.1692785779996484
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 5 ++++++++++++++++++
Elapsed time:  0.16060877699965204
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 6 ++++++++++++++++++
Elapsed time:  0.1585121080006502
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_0] | 7 ++++++++++++++++++
Elapsed time:  0.1581562400006078
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_1_0_12,0.927641,0.713076,0.628731,0.614951,0.622360,0.303676,1.918660,0.651213,1.036919,0.844066
model_1_0_11,0.927422,0.712662,0.628304,0.614683,0.622032,0.304592,1.921430,0.651961,1.037640,0.844801
model_1_0_10,0.927199,0.712242,0.627870,0.614408,0.621695,0.305529,1.924237,0.652723,1.038383,0.845553
model_1_0_9,0.926972,0.711817,0.627429,0.614125,0.621351,0.306484,1.927083,0.653497,1.039145,0.846321
model_1_0_8,0.926739,0.711385,0.626980,0.613834,0.621000,0.307460,1.929969,0.654284,1.039928,0.847106
model_1_0_7,0.926502,0.710948,0.626523,0.613534,0.620640,0.308456,1.932894,0.655085,1.040736,0.847911
model_1_0_6,0.926260,0.710504,0.626060,0.613226,0.620273,0.309473,1.935859,0.655897,1.041565,0.848731
model_1_0_5,0.926012,0.710055,0.625589,0.612910,0.619898,0.310510,1.938866,0.656724,1.042416,0.849570
model_1_0_4,0.925760,0.709599,0.625109,0.612585,0.619514,0.311570,1.941914,0.657565,1.043291,0.850428
model_1_0_3,0.925502,0.709137,0.624622,0.612251,0.619121,0.312651,1.945004,0.658420,1.044191,0.851306


DataFrame salvo em ../results/metrics_1_0
+++++++++++ [1_1] | 1 ++++++++++++++++++
Elapsed time:  14.979221934001544
Stopped at epoch:  628
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_1] | 2 ++++++++++++++++++
Elapsed time:  0.4425085590009985
Stopped at epoch:  17
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_1] | 3 ++++++++++++++++++
Elapsed time:  0.35593181399963214
Stopped at epoch:  13
1/1 [==============================] - 0s 19ms/step
+++++++++++ [1_1] | 4 ++++++++++++++++++
Elapsed time:  0.7186993220002478
Stopped at epoch:  32
1/1 [==============================] - 0s 13ms/step
+++++++++++ [1_1] | 5 ++++++++++++++++++
Elapsed time:  0.6755512700001418
Stopped at epoch:  13
1/1 [==============================] - 0s 20ms/step
+++++++++++ [1_1] | 6 ++++++++++++++++++
Elapsed time:  0.2092648680009006
Stopped at epoch:  8
1/1 [==============================] - 0s 14ms/step
+++++++++++ [1_1] | 7 ++++++++++++++++++
Elapsed time:  0.37378

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_1_1_24,0.939033,0.746793,0.636204,0.975863,0.913735,0.255864,1.693193,1.417251,0.181899,0.799575
model_1_1_23,0.939035,0.746792,0.636219,0.975862,0.913737,0.255856,1.693201,1.417195,0.181907,0.799550
model_1_1_22,0.939037,0.746791,0.636234,0.975861,0.913740,0.255848,1.693211,1.417137,0.181915,0.799526
model_1_1_21,0.939037,0.746791,0.636235,0.975861,0.913740,0.255848,1.693212,1.417131,0.181916,0.799524
model_1_1_20,0.939037,0.746791,0.636237,0.975861,0.913741,0.255847,1.693213,1.417125,0.181917,0.799521
model_1_1_19,0.939038,0.746790,0.636238,0.975861,0.913741,0.255846,1.693214,1.417118,0.181917,0.799518
model_1_1_18,0.939038,0.746790,0.636240,0.975861,0.913741,0.255845,1.693215,1.417111,0.181918,0.799515
model_1_1_17,0.939038,0.746790,0.636242,0.975861,0.913742,0.255844,1.693216,1.417104,0.181919,0.799511
model_1_1_16,0.939038,0.746790,0.636244,0.975861,0.913742,0.255843,1.693217,1.417097,0.181920,0.799508
model_1_1_15,0.939039,0.746790,0.636246,0.975860,0.913742,0.255842,1.693218,1.417089,0.181920,0.799505


...

DataFrame salvo em ../results/metrics_17_8
+++++++++++ [17_9] | 1 ++++++++++++++++++
Elapsed time:  0.1722766960010631
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [17_9] | 2 ++++++++++++++++++
Elapsed time:  0.3449660020014562
Stopped at epoch:  6
1/1 [==============================] - 0s 19ms/step
+++++++++++ [17_9] | 3 ++++++++++++++++++
Elapsed time:  0.16698169499795767
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [17_9] | 4 ++++++++++++++++++
Elapsed time:  0.17135651300122845
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [17_9] | 5 ++++++++++++++++++
Elapsed time:  0.1718012719975377
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [17_9] | 6 ++++++++++++++++++
Elapsed time:  0.1579967180005042
Stopped at epoch:  6
1/1 [==============================] - 0s 13ms/step
+++++++++++ [17_9] | 7 ++++++++++++++++++
Elapsed time:  0.15

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_17_9_0,0.966814,0.774572,0.980750,0.952093,0.961524,0.139273,1.507436,0.065292,0.340910,0.203101
model_17_9_1,0.966866,0.774269,0.981498,0.950627,0.960777,0.139057,1.509467,0.062753,0.351336,0.207045
model_17_9_2,0.966893,0.773948,0.982156,0.949142,0.959987,0.138941,1.511609,0.060522,0.361906,0.211214
model_17_9_3,0.966897,0.773612,0.982725,0.947637,0.959155,0.138926,1.513858,0.058592,0.372618,0.215605
model_17_9_4,0.966878,0.773260,0.983208,0.946112,0.958282,0.139008,1.516210,0.056955,0.383470,0.220212
model_17_9_5,0.966835,0.772894,0.983606,0.944567,0.957369,0.139186,1.518660,0.055605,0.394460,0.225032
model_17_9_6,0.966771,0.772513,0.983921,0.943004,0.956417,0.139457,1.521205,0.054534,0.405584,0.230059
model_17_9_7,0.966684,0.772119,0.984157,0.941422,0.955426,0.139820,1.523840,0.053736,0.416841,0.235289
model_17_9_8,0.966576,0.771712,0.984313,0.939822,0.954398,0.140272,1.526563,0.053204,0.428228,0.240716
model_17_9_9,0.966448,0.771292,0.984394,0.938204,0.953333,0.140811,1.529369,0.052930,0.439743,0.246336


DataFrame salvo em ../results/metrics_17_9
Testando combinacao18: Hidden Size=[6, 3], regularizer=0.5, learning_rate=0.1
+++++++++++ [18_0] | 1 ++++++++++++++++++
Elapsed time:  0.45013556699996116
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [18_0] | 2 ++++++++++++++++++
Elapsed time:  0.23808760899919434
Stopped at epoch:  7
1/1 [==============================] - 0s 21ms/step
+++++++++++ [18_0] | 3 ++++++++++++++++++
Elapsed time:  0.4458915290015284
Stopped at epoch:  6
1/1 [==============================] - 0s 19ms/step
+++++++++++ [18_0] | 4 ++++++++++++++++++
Elapsed time:  0.44356035199962207
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [18_0] | 5 ++++++++++++++++++
Elapsed time:  0.2160081469992292
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [18_0] | 6 ++++++++++++++++++
Elapsed time:  0.21482898799877148
Stopped at epoch:  8
1/1 [============================

,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt
model_18_7_1,0.944751,0.705774,0.945367,0.840334,0.931486,0.231870,1.967492,0.326979,0.290539,0.308759
model_18_7_2,0.939123,0.702381,0.920377,0.759128,0.898497,0.255489,1.990183,0.476538,0.438305,0.457421
model_18_7_0,0.947480,0.702228,0.970095,0.895108,0.958965,0.220413,1.991203,0.178977,0.190868,0.184923


DataFrame salvo em ../results/metrics_18_7
+++++++++++ [18_8] | 1 ++++++++++++++++++
Elapsed time:  0.16618294499858166
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [18_8] | 2 ++++++++++++++++++
Elapsed time:  0.1606712510001671
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [18_8] | 3 ++++++++++++++++++
Elapsed time:  0.21671955300189438
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [18_8] | 4 ++++++++++++++++++
Elapsed time:  0.1631459790005465
Stopped at epoch:  6
1/1 [==============================] - 0s 15ms/step
+++++++++++ [18_8] | 5 ++++++++++++++++++
Elapsed time:  0.16276920300151687
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [18_8] | 6 ++++++++++++++++++
Elapsed time:  0.1572036899997329
Stopped at epoch:  6
1/1 [==============================] - 0s 14ms/step
+++++++++++ [18_8] | 7 ++++++++++++++++++
Elapsed time:  0.1